<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
pd.set_option("mode.copy_on_write", True)

import os
os.chdir("/Users/paytonrodman/projects/migraine_weather")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<h1>Creating a Map of Pressure Variation</h1>

In [3]:
import country_converter as coco
import meteostat
from datetime import datetime
from migraine_weather.data import clean_data, calculate_variation

country_code = 'AU'
start = datetime(2000, 1, 1, 0, 0, 0)
end = datetime(2020, 12, 31, 23, 59, 59)
stations = meteostat.Stations().region(country_code).fetch()
stations = stations[(stations['hourly_start'].dt.year<=start.year) & (stations['hourly_end'].dt.year>=end.year)]

n_stations = len(stations)
current_n = 0
av_frac_var = []
for station_id in stations.index:
    current_n += 1
    station_name = stations[stations.index==station_id]['name'].iloc[0]
    #print(station_name)
    print(f'\r{(current_n/n_stations)*100:.2f}%', end='', flush=True)
    
    df = meteostat.Hourly(station_id, start, end).fetch()
    # if pressure measurements are NaN, skip
    if (df["pres"].isna().all()) | (len(df[df["pres"].isna()])>0.1*len(df["pres"])):
        av_frac_var.append(float('nan'))
        continue
    
    cleaned_df = clean_data.remove_outliers(df)
    frac_var_yearly = calculate_variation.get_variation_frac(cleaned_df, "pres")
    av_frac_var.append(sum(frac_var_yearly.values())/len(frac_var_yearly)) # average fraction of days with high variation per year
    
stations.insert(0, "frac_var", av_frac_var)
stations = stations.drop(stations[stations['frac_var'] == 'NaN'].index)
print(stations)

ModuleNotFoundError: No module named 'dotenv'

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# initialize an axis
fig, ax = plt.subplots(figsize=(12,12))
countries = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
countries[countries["name"] == country_name].plot(color="lightgrey", ax=ax)

stations.plot(x="longitude", y="latitude", kind="scatter", c="frac_var", colormap="YlOrRd", ax=ax)
plt.show()